# Hyperparameter Tuning for ChatGPT Query Reduction

In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
ensure_pyterrier_is_loaded()

import pyterrier as pt
import os
import sys
import pandas as pd
from tqdm import tqdm
sys.path.append('chat-gpt')
from chatgpt_query_reduction import ChatGPTQueryReduction


index = pt.IndexRef.of(os.path.abspath('pyterrier-index/index/data.properties'))

In [3]:
def report_effectiveness(prompt, dataset):
    queries = pt.io.read_topics(f'tot-{dataset}/queries.xml', 'trecxml')
    chat_gpt_query_reduction = ChatGPTQueryReduction('chat-gpt/query-variants-in-progress', prompt).as_transformer()

    bm25 = pt.BatchRetrieve(index, wmodel="BM25")

    retrieval_pipeline = chat_gpt_query_reduction >> bm25

    ret = pt.Experiment([retrieval_pipeline], queries, pt.io.read_qrels(f'tot-{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10']}

def report_pipeline(prompt):
    ret = {'Prompt': prompt}
    for dataset in ['train', 'dev']:
        for k,v in report_effectiveness(pipeline, dataset).items():
            ret[dataset + '_' + k] = v
    return ret

df = []
for pipeline in tqdm(['1', '2', '3']):
    df += [report_pipeline(pipeline)]

df = pd.DataFrame(df)
df.to_json('hyperparameter-tuning-chat-gpt-results.jsonl', lines=True, orient='records')

100%|██████████| 3/3 [01:37<00:00, 32.62s/it]


In [6]:
df = pd.read_json('hyperparameter-tuning-chat-gpt-results.jsonl', lines=True)
df

,Prompt,train_ndcg_cut_10,train_recip_rank,train_P_10,dev_ndcg_cut_10,dev_recip_rank,dev_P_10
0,1,0.109710,0.108053,0.014000,0.110954,0.107583,0.014667
1,2,0.099082,0.093880,0.014000,0.083042,0.078846,0.011333
2,3,0.071147,0.067398,0.010667,0.113501,0.104607,0.015333


### Small Example

In [3]:
chatgpt_query_reduction = ChatGPTQueryReduction(f'chat-gpt/query-variants-in-progress', '1').as_transformer()
queries = pt.io.read_topics(f'tot-train/queries.xml', 'trecxml')


In [4]:
pd.set_option('display.max_colwidth', None)
chatgpt_query_reduction(queries)

,qid,query_0,query
0,763,super rare surreal dystopian masterpiece very rare movie that is scifi dystopian experimental surreal it s like stalker meets el topo meets holy mountain meets alphaville meets delicatessen meets hard to be a god like kurosawa tarkovsky and lynch had a kid together it was color possibly russian and i don t really remember the decade but want to say 60s or 70s though could easily be more recent it is very rare there is only one crappy partial print of it and that is what the youtube version is from lot of wide shots in a surreal wilderness winter settings strange bleeding saturation in some shots crazy costumes seriously one of the strangest films i ve ever seen and my favorite films are strange weird ones if you ve ever seen what you re thinking of on a best weird movies or you ve never seen this list that s not it i don t think this film even has a cult following of ten people it s an actual rare gem have been looking through selections at 366 weird movies and not found it yet btw the way most of those titles are exactly the kind of not actually rare movies this film is definitely not,rare sci fi dystopian experimental surreal film hard to find only one poor print available possibly russian 60s 70s features wide shots in surreal wilderness winter settings and unique costumes one of the strangest films ever seen unlike those found on best weird movies or you ve never seen this lists actual rare gem not part of a cult following not found on 366 weird movies selection
1,802,male little person falls in love with red headed woman i hope someone can help me track down this movie i may have some details mixed up or combined with another drama so if you have any suggestions at all please share them there is a caucasian man who is a little person who is successful in his working life but is lonely in his private life he may be a writer or editor or some such professional i think part of the plot is that he might have trouble making friends because some people are prejudiced against his size sometimes he abuses alcohol because of this he re connects with a woman he knew when he was a child even though she used to tease him back when they were children they become friends and eventually fall in love she is a regular height woman also caucasian slim and with red hair in the last scene of the movie they are dancing on a balcony or rooftop and she is wearing a white dress maybe it s their wedding i have been through the filmographies of all the male caucasian little people actors i can think of with no luck my googlefu has failed me as well if you have any ideas at all no matter how far fetched please share them thank you in advance for any suggestions you may have,male little person falls in love with red headed woman the man is successful in his career but struggles with loneliness and prejudice due to his size he reconnects with a woman from his childhood who used to tease him they become friends and eventually fall in love the movie ends with them dancing on a balcony or rooftop possibly their wedding please help me find this movie
2,950,movie about two girls who run away and murder an old couple in bed have sex on the same bloodied bed i m not sure if this is even a movie a tv show or if i dreamt it or imagined it the scene that i most vividly remember is where two teenage early 20s girls knock on the door of an elderly couple the couple lets them in and while the couple is asleep the girls brutally murder them and then proceed to have sex while in the same bloodied bed as the murdered old couple another scene is where one girl is in the shower drinking beer and the other girl joins her and the have sex in the shower another scene which i m not sure if it s even part of the movie or not the girls break in to an old lady s trailer home and molest her wear her underwear and then a cop finds them and kills them not sure if that last scene is correct at all i saw this movie maybe around 1999 it was in color and it